### Procedure

Matching of the candidates for entities to find in the question is as follows:
1) lower text and find exact matching
2) lower text and find the main part of the candidate without the part in brackets
3) lower text and try to find it using fuzzywuzzy package (count for typos, misprints, etc)

After all steps (1,2,3) auto_ner function checks the matching for beqing adequate (if the length of the found entity is less then 2, then matching is wrong)

In the end for all sentences where there is no matching, "start" token is set at the beginning of the sentence while "end" token in the end

In [1]:
! pip install fuzzywuzzy

In [140]:
import pandas as pd
from tqdm import tqdm
from fuzzywuzzy import fuzz
import numpy as np

In [153]:
data = pd.read_csv('data_third_iteration_val.csv')

In [154]:
for j in range(len(data)):
    if data.loc[j, 'subject_text_all'] == 'Nan':
        data.loc[j, 'subject_text_all'] = np.nan

### Auto NER with check for good NER retrieving

In [155]:
#Auto NER

def auto_ner(data):
    for k in tqdm(range(len(data))):
        if pd.notna(data.loc[k, 'question_ner_auto']):
            data.loc[k, 'ner_auto'] = data.loc[k, 'question_ner_auto'].split('[START] ')[1].split(' [END]')[0]
    for k in tqdm(range(len(data))):
        if pd.notna(data.loc[k, 'question_ner_auto']):
            data.loc[k, 'ner_auto'] = " ".join(data.loc[k, 'ner_auto'].split())
    for k in tqdm(range(len(data))):
        if pd.notna(data.loc[k, 'ner_auto']):
            if len(data.loc[k, 'ner_auto']) < 2:
                data.loc[k, 'question_ner_auto'] = np.nan
                data.loc[k, 'ner_auto'] = np.nan
    return data

### Start-End Tokens placing

In [156]:
#general module (direct correspondance)

for i in tqdm(range(len(data))):
    init_question = data.loc[i, 'question']
    cur_question =  data.loc[i, 'question'].lower()
    if pd.notna(data.loc[i, 'subject_text_all']):
        candidates = data.loc[i, 'subject_text_all'].lower()
        for cand in candidates.split(','):
            if cand in cur_question:
                ind = cur_question.find(cand)
                new_ques = init_question[:ind]+ '[START] '+ init_question[ind:ind+len(cand)] + ' [END]' + init_question[ind+len(cand):]
                data.loc[i, 'question_ner_auto'] = new_ques
                break

100%|██████████| 2821/2821 [00:00<00:00, 10083.28it/s]


In [157]:
data = auto_ner(data)

100%|██████████| 2821/2821 [00:00<00:00, 66119.02it/s]


In [158]:
#cut the part in the brackets

for i in tqdm(range(len(data))):
    init_question = data.loc[i, 'question']
    cur_question =  data.loc[i, 'question'].lower()
    if pd.notna(data.loc[i, 'subject_text_all']):
        candidates = data.loc[i, 'subject_text_all'].lower()
        if pd.isna(data.loc[i, 'question_ner_auto']):
            for cand in candidates.split(','):
                ind_brack = cand.find('(')
                newcand = cand[:ind_brack-1]
                if newcand in cur_question:
                    ind = cur_question.find(newcand)
                    new_ques = init_question[:ind]+ '[START] '+ init_question[ind:ind+len(newcand)] + ' [END]' + init_question[ind+len(newcand):]
                    data.loc[i, 'question_ner_auto'] = new_ques
                    break

100%|██████████| 2821/2821 [00:00<00:00, 18960.63it/s]


In [159]:
data = auto_ner(data)

100%|██████████| 2821/2821 [00:00<00:00, 60165.11it/s]


In [160]:
#fuzzywuzzy part

for i in tqdm(range(len(data))):
    init_question = data.loc[i, 'question']
    cur_question =  data.loc[i, 'question'].lower()
    if pd.notna(data.loc[i, 'subject_text_all']):
        candidates = data.loc[i, 'subject_text_all'].lower()

        # find substring with similar amount of words by fuzz ratio
        if pd.isna(data.loc[i, 'question_ner_auto']):
            curr_fuzz_ratio = 0
            curr_substring = 'test'
            for cand in candidates.split(','):
                len_target = len(cand.split(" "))
                for j in range(len(cur_question.split(" "))-len_target+1):
                    substring = ' '.join(cur_question.split(" ")[j:j+len_target])
                    if fuzz.ratio(substring, cand) >= 80 and fuzz.ratio(substring,cand) > curr_fuzz_ratio:
                        curr_fuzz_ratio = fuzz.ratio(substring, cand)
                        curr_substring = substring

                if curr_fuzz_ratio > 0:
                    ind1 = cur_question.find(curr_substring)
                    ind2 = ind1+len(curr_substring)
                    new_ques = init_question[:ind1]+ '[START] '+init_question[ind1:ind2]+ ' [END]' + init_question[ind2:]
                    data.loc[i, 'question_ner_auto'] = new_ques
                    break
        # find substring with similar start and end words by fuzz ratio
        if pd.isna(data.loc[i, 'question_ner_auto']):
            for cand in candidates.split(','):
                inds = []
                for elem in cand.split(' '):
                    questions_words = cur_question.split(' ')
                    for word in questions_words:
                        if fuzz.ratio(word, elem) >= 75:
                            inds.append(cur_question.find(word))
                            inds.append(cur_question.find(word)+len(word))
                inds = sorted(list(set(inds)))
                try:
                    new_ques = init_question[:inds[0]]+ '[START] '+init_question[inds[0]:inds[-1]]+ ' [END]' + init_question[inds[-1]:]
                    data.loc[i, 'question_ner_auto'] = new_ques
                except:
                    continue
                break

        # find substring where substring is without white space by fuzz ratio
        if pd.isna(data.loc[i, 'question_ner_auto']):
            for cand in candidates.split(','):
                sub = ''.join(cand.split(' '))
                if sub in cur_question:
                    ind1 = cur_question.find(sub)
                    ind2 = ind1 + len(substring)
                    new_ques = init_question[:ind1]+ '[START] '+init_question[ind1:ind2]+ ' [END]' + init_question[ind2:]
                    data.loc[i, 'question_ner_auto'] = new_ques


100%|██████████| 2821/2821 [00:00<00:00, 10554.39it/s]


In [161]:
data = auto_ner(data)

100%|██████████| 2821/2821 [00:00<00:00, 59064.97it/s]


### Checks of matching

In [162]:
data[data['question_ner_auto'].isna()]

,subject,property,object,question,subject_text,flag,subject_text_add,subject_text_all,question_ner_auto,ner_auto
75,Q579903,P264,Q1047366,Which label is sabrinawmusic signed to?,"Sabrina Washington, OMG (Oh My Gosh), Puma Was...",0.0,NaN,"Sabrina Washington, OMG (Oh My Gosh), Puma Was...",NaN,NaN
223,Q1892868,P264,Q3489789,what record label worked with tormentandtoreros,"Marc and the Mambas, Marc &amp; The Mambas, Ma...",0.0,NaN,"Marc and the Mambas, Marc &amp; The Mambas, Ma...",NaN,NaN
320,Q1677192,P21,Q6581097,What is jackrosekensington gender?,Jack Rose (guitarist),1.0,NaN,Jack Rose (guitarist),NaN,NaN
342,Q8773673,P364,Q9043,what is the language of the crossing?,Andreaskorset,1.0,NaN,Andreaskorset,NaN,NaN
350,Q10339444,P136,Q93204,what kind of movie is the mystery of samba,Nan,1.0,NaN,NaN,NaN,NaN
379,Q4892194,P21,Q6581072,What is the gender of the fictional character ...,Nan,1.0,NaN,NaN,NaN,NaN
490,Q49085,R172,Q3807984,What ethnicity is Jermaine Jackson?,"African Americans, Africa American, African Am...",0.0,NaN,"African Americans, Africa American, African Am...",NaN,NaN
638,Q770275,P21,Q6581097,what is john mark karr's gender,Alexis Reich,1.0,NaN,Alexis Reich,NaN,NaN
706,Q3686120,P136,Q93204,What genre of movie is concerto per michelangelo?,Nan,1.0,NaN,NaN,NaN,NaN
1198,Q635813,P136,Q132438,what type of music is godthelux,"Vesania, Vesania (band)",0.0,NaN,"Vesania, Vesania (band)",NaN,NaN


### Add Start-End Tokens for sentences where NER was not found

In [163]:
for j in range(len(data)):
    if pd.isna(data.loc[j, 'question_ner_auto']):
        data.loc[j, 'question_ner_auto'] = '[START] '+str(data.loc[j, 'question'])+ '[END]'

In [164]:
data.to_csv('val_ner_auto.csv', index = False)